In [4]:
import mlflow
from mlflow import get_experiment_by_name

from hcve_lib.metrics import StratifiedMetric
from hcve_lib.utils import notebook_init, transpose_dict, transpose_list

notebook_init()
from deps.data import get_30_to_80
from hcve_lib.tracking import load_run_results
from deps.constants import RANDOM_STATE
from hcve_lib.evaluation_functions import compute_metric_result, merge_predictions, average_group_scores
from hcve_lib.metrics import BootstrappedMetric, CIndex

from mlflow import set_tracking_uri
import pandas
from pandas import DataFrame, Series
from toolz import valmap

set_tracking_uri('http://localhost:5000')
from numpy import mean

from typing import Dict, List
from plotly import express as px
from deps.evaluation import get_latest_chosen_run_group

from hcve_lib.functional import star_args
from toolz import itemmap
from statistics import mean
from hcve_lib.functional import dict_subset

from numpy import std
from deps.visualisation import plot_metric_matrix
from deps.evaluation import get_metric_matrix
from hcve_lib.tracking import load_group_results
from deps.visualisation import get_numerical_matrix
from deps.visualisation import plot_matrix
from config import GROUPS_LCO, GROUPS_LM_SELECTED, GROUPS_10_fold
from config import COHORTS

%autoreload 2
ITERATIONS = 3

<frozen importlib._bootstrap>:228: RuntimeWarning: sklearn.tree._splitter.Splitter size changed, may indicate binary incompatibility. Expected 360 from C header, got 1160 from PyObject


In [5]:
from deps.common import get_data_cached

data, metadata, X, y = get_data_cached()


[Memory]1.0s, 0.0min    : Loading get_data...
____________________________________________get_data cache loaded - 0.0s, 0.0min


In [11]:

merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id, load_models=False))),
    GROUPS_LCO,
)

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/pandas/_libs/internals.cpython-39-x86_64-linux-gnu.so'>

In [2]:
from hcve_lib.data import binarize_event

def plot_y_score_dist(y_binarized, merged_predictions):
    for name, prediction in merged_predictions.items():
        fig = px.violin(DataFrame({
            'y_score': prediction['y_score'].loc[y_binarized.index],
            'y_true': y_binarized,
        }).dropna(), y='y_score', x='y_true', points='all', title=name)
        fig.show()
    
plot_y_score_dist(binarize_event(10*365, y['data']), merged_prediction)

NameError: name 'y' is not defined

In [12]:
result_lm = average_group_scores(load_group_results('1748b866-20a3-43a2-ac1b-93e41daaca2c'))

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/toolz/dicttoolz.py:83: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  rv.update(zip(d.keys(), map(func, d.values())))


In [13]:
metrics_seed_averaged_lm = valmap(
    lambda group_id: compute_metric_result(
        BootstrappedMetric(
            CIndex(),
            random_state=RANDOM_STATE,
            iterations=10,
        ),
        y,
        average_group_scores(load_group_results(group_id)),
    ),
    GROUPS_LM_SELECTED,
)

/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/toolz/dicttoolz.py:83: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  rv.update(zip(d.keys(), map(func, d.values())))


In [14]:
from collections import defaultdict


metrics_lm_averaged = defaultdict(dict)
for method_name, values in metrics_seed_averaged_lm.items():
    for test_cohort in COHORTS:
        values_selected = [value['mean'] for split, value in values.items() if split[1] == test_cohort]
        metrics_lm_averaged[method_name][test_cohort] = mean(values_selected)


In [16]:
from hcve_lib.functional import try_except

metrics_seed_averaged_lco = valmap(
    lambda run_id: try_except(lambda: compute_metric_result(
            BootstrappedMetric(
                CIndex(),
                random_state=RANDOM_STATE,
                iterations=10,
            ),
            y,
            average_group_scores(load_group_results(run_id)),
        ),
        {AttributeError: lambda e: None}
    ),
    GROUPS_LCO,
)

In [53]:
del metrics_seed_averaged_lco['xgb']
del metrics_seed_averaged_lco['dcm']


KeyError: 'xgb'

In [54]:
from hcve_lib.functional import pipe, flatten


metrics_seed_averaged_df = pipe([
    [
        {
            'method_name': method_name,
            'test_cohort': split_name,
            'mean': metric['mean'],
            'ci_hi': metric['ci'][1]-metric['mean'],
        }
        for split_name, metric in result.items()
    ]
    for method_name, result in metrics_seed_averaged_lco.items()
],
    flatten,
    list,
    DataFrame,
)

metrics_seed_averaged_df_selected = metrics_seed_averaged_df[
    metrics_seed_averaged_df['method_name'].isin(['gb', 'coxnet'])
]
metrics_seed_averaged_df_selected['splits'] = 'Multi-center'

In [55]:
metrics_seed_averaged_lm = {'method_name': [], 'mean': [], 'test_cohort': [], 'splits': []}

for method_name, metrics_by_test in metrics_lm_averaged.items():
    metrics_seed_averaged_lm['method_name'] += ([method_name] * len(metrics_by_test))
    metrics_seed_averaged_lm['splits'] += (['Single-center'] * len(metrics_by_test))
    metrics_seed_averaged_lm['test_cohort'] += list(metrics_by_test.keys())
    metrics_seed_averaged_lm['mean'] += list(metrics_by_test.values())

In [56]:
lco_lm_comparison = pandas.concat([DataFrame(metrics_seed_averaged_lm), metrics_seed_averaged_df_selected])

In [68]:
from hcve_lib.visualisation import setup_plotly_style

labels={
     "mean": "c-index",
    'test_cohort': 'Tested on'
 }
lco_lm_comparison_coxnet = lco_lm_comparison[lco_lm_comparison['method_name'] == 'coxnet']

fig = px.bar(
    lco_lm_comparison_coxnet,
    x='test_cohort',
    y='mean',
    barmode='group',
    range_y=[0.5,0.8],
    labels=labels,
    pattern_shape='splits',
)
fig.update_traces(marker_color='#4194D9')
setup_plotly_style(fig)
fig.write_image('./output/coxnet_lm_lco.svg')
fig.show()

lco_lm_comparison_gb = lco_lm_comparison[lco_lm_comparison['method_name'] == 'gb']

fig = px.bar(
    lco_lm_comparison_gb,
    x='test_cohort',
    y='mean',
    color_continuous_scale='color',
    barmode='group',
    range_y=[0.5,0.8],
    pattern_shape='splits',
    labels=labels,
)
fig.update_traces(marker_color='#FF7F00')
setup_plotly_style(fig)
fig.write_image('./output/gb_lm_lco.svg')
fig.show()

In [ ]:
def compute_basic_statistics(method_name, metric_values):
    values = list(metric_values.values())
    return method_name, (mean(values), std(values))

### C-index: average test metrics
(std is not so informative here, requires a different method)a

In [ ]:
metrics_seed_averaged

**Average per method (LCO)**

In [ ]:
averaged_metrics = itemmap(star_args(compute_basic_statistics), metrics_seed_averaged)
averaged_metrics

**Average per method (10-fold)**

In [ ]:
averaged_metrics = itemmap(star_args(compute_basic_statistics), metrics_seed_averaged_10_fold)
averaged_metrics

### C-index: merge test sets (LCO)

(std is not so informative here, requires a different method)a

In [ ]:
merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)

metrics_merged = compute_metric_result(
    CIndex(),
    y,
    merged_prediction,
)

In [ ]:
metrics_merged

### C-index: merge test sets (10-fold)


In [ ]:
merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS_10_fold,
)

metrics_merged = compute_metric_result(
    CIndex(),
    y,
    merged_prediction,
)

metrics_merged

### Number individuals removed by 30-80 age filter

In [ ]:
from hcve_lib.visualisation import setup_plotly_style

print(len(X) - len(get_30_to_80(X)))

### Merged boxplot (weighted)

In [ ]:
get_merge_boxplot_metrics

In [66]:

from config import *


def get_merge_boxplot_metrics(groups, random_state=RANDOM_STATE, weighted: bool = True, return_summary=False):
    merged_prediction = valmap(
        lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
        groups,
    )
    merged_prediction = {method: {**prediction, 'y_score': prediction['y_score'].loc[data.index]} for method, prediction in merged_prediction.items()}    
    
    return valmap(
        lambda result: compute_metric_prediction(
            BootstrappedMetric(
                WeightedCIndex(weight=map_inverse_weight(data['STUDY']) if weighted else None),
                random_state,
                iterations=5,
                return_summary=return_summary,
            ),
            y,
            result,
        ),
        merged_prediction,
    )



In [ ]:
merged_metrics = get_merge_boxplot_metrics(GROUPS)

In [ ]:
merged_metrics_summary = get_merge_boxplot_metrics(GROUPS, return_summary=True)

In [ ]:
merged_metrics_summary

In [ ]:
from plotly.graph_objs import *
from config import COLORS


def plot_merged_metrics(merged_metrics):

    
    metrics_df = DataFrame()

    for method_name, method_metric_values in merged_metrics.items():
        metric_values = method_metric_values["c_index"]
        metrics_df_add = DataFrame({
            'method': [METHODS.get(method_name, method_name)] * len(metric_values),
            'method_original': [method_name] * len(metric_values),
            'values': metric_values,
            'color': [COLORS[method_name]] * len(metric_values),
        })
        metrics_df = pandas.concat([metrics_df, metrics_df_add])

    fig = px.violin(
        metrics_df,
        x="method_original",
        y="values",
        color="method_original",
        range_y=[0.65, 0.8],
        labels={'method': 'Test', 'values': 'c-index', **METHODS},
        box=True,
        color_discrete_map=COLORS,
    )
    setup_plotly_style(fig)
    fig.update_traces(marker_size=2, marker_opacity=0.9)
    fig.update_layout(legend=dict(itemsizing='constant'))

#     fig.update_xaxes(
#         tickfont=dict(size=25)
#     )

    fig.update_layout(
        font=dict(family=FONT, size=28),
        bargroupgap=0.1,
        template='simple_white',
        xaxis_ticktext=metrics_df["method"]
    )
    
    fig.update(
        layout_showlegend=False,
        layout_annotations=[
            dict(
                x=i+0.4,
                y=(m_c := mean(metrics['c_index'])),
                text=f'{m_c:.3f}',
                showarrow=False,
                font=dict(
                    size=25,
                    color=COLORS[method_name],
                )
            )
            for i, (method_name, metrics) in enumerate(merged_metrics_selected.items())
        ],
    )

    return fig

merged_metrics_selected = dict_subset(('coxnet', 'gb'), merged_metrics)

fig = plot_merged_metrics(merged_metrics_selected)
fig.write_image('./data/main_c_index.svg')
fig.show()

In [ ]:
from hcve_lib.metrics import WeightedCIndex
from hcve_lib.evaluation_functions import map_inverse_weight

merged_metrics_class_weighted = valmap(
    lambda result: compute_metric_prediction(
        BootstrappedMetric(
            WeightedCIndex(weight=map_inverse_weight(y['data']['label'])),
            RANDOM_STATE,
            iterations=ITERATIONS,
            return_summary=False,
        ),
        y,
        result,
    ),
    merged_prediction,
)


In [ ]:
plot_merged_metrics(merged_metrics_class_weighted)

### C-index: merged y_scores (unweighted)

In [ ]:
metrics_df = DataFrame()
for method_name, method_metric_values in merged_metrics.items():
    metric_values = method_metric_values["c_index"]
    metrics_df_add = DataFrame({
        'method': [method_name] * len(metric_values),
        'values': metric_values,
    })
    metrics_df = pandas.concat([metrics_df, metrics_df_add])

fig = px.strip(metrics_df, x="method", y="values", stripmode='overlay', range_y=[0.5, 1])
fig.update_traces(marker_size=2, marker_opacity=0.9)
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()

In [ ]:


merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)

merged_prediction['pcp_hf'] = merge_predictions(load_run_results('229ef6c05f5d40acb57f5f62707f1e38'))

merged_metrics = valmap(
    lambda result: compute_metric_prediction(
        BootstrappedMetric(
            StratifiedMetric(
                CIndex(),
                splits={'30_to_80': list(get_30_to_80(X).index)}
            ),
            RANDOM_STATE,
            iterations=ITERATIONS,
            return_summary=False,
        ),
        y,
        result,
    ),
    merged_prediction,
)


In [ ]:
metrics_df = DataFrame()
for method_name, method_metric_values in merged_metrics.items():
    metric_values = method_metric_values["30_to_80__c_index"]
    metrics_df_add = DataFrame({
        'method': [method_name] * len(metric_values),
        'values': metric_values,
    })
    metrics_df = pandas.concat([metrics_df, metrics_df_add])

In [ ]:
fig = px.strip(metrics_df, x="method", y="values", stripmode='overlay', range_y=[0.5, 1])
fig.update_traces(marker_size=2, marker_opacity=0.9)
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()

In [ ]:
from hcve_lib.evaluation_functions import compute_metric_prediction

merged_prediction = valmap(
    lambda group_id: merge_predictions(average_group_scores(load_group_results(group_id))),
    GROUPS,
)

merged_metrics = valmap(
    lambda result: compute_metric_prediction(
        BootstrappedMetric(
            StratifiedMetric(
                CIndex(),
                splits={'30_to_80': list(get_30_to_80(X).index)}
            ),
            RANDOM_STATE,
            iterations=ITERATIONS,
            return_summary=False,
        ),
        y,
        result,
    ),
    merged_prediction,
)


In [ ]:
metrics_df = DataFrame()
for method_name, method_metric_values in merged_metrics.items():
    metric_values = method_metric_values["30_to_80__c_index"]
    metrics_df_add = DataFrame({
        'method': [method_name] * len(metric_values),
        'values': metric_values,
    })
    metrics_df = pandas.concat([metrics_df, metrics_df_add])

In [ ]:
fig = px.strip(metrics_df, x="method", y="values", stripmode='overlay', range_y=[0.5, 1])
fig.update_traces(marker_size=2, marker_opacity=0.9)
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()

### Predictive performance: averaged y_scores

In [ ]:
averaged_results = {}


In [ ]:
metrics_per_cohort_df = DataFrame()

for method_name, method_metric_values in metrics_per_cohort.items():
    for test_cohort_name, metric_values in method_metric_values.items():
        try:
            metrics_df_add = DataFrame({
                'method': [method_name] * len(metric_values),
                'test_cohort': [test_cohort_name] * len(metric_values),
                'values': metric_values,
            })
            metrics_per_cohort_df = pandas.concat([metrics_per_cohort_df, metrics_df_add])
        except TypeError:
            print('ERROR')

In [ ]:
fig = px.strip(metrics_per_cohort_df, x="method", y="values", color='test_cohort', stripmode='overlay',
               range_y=[0.5, 1])
fig.update_traces(marker_size=2, marker_opacity=0.9)
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()

### Bar chart predictive performance

In [ ]:
bar_chart_results = {}


In [ ]:
for group_name, group_id in GROUPS.items():
    group = load_group_results(group_id)
    bar_chart_results[group_name] = average_group_scores(group)

bar_chart_results['pcp_hf'] = load_run_results('229ef6c05f5d40acb57f5f62707f1e38')

metrics_per_cohort = valmap(
    lambda prediction: compute_metric_result(
        BootstrappedMetric(
            StratifiedMetric(CIndex(), splits={'30_to_80': list(get_30_to_80(X).index)}),
            RANDOM_STATE,
            iterations=5,
            return_summary=True,
        ),
        y,
        prediction,
    ),
    bar_chart_results,
)

In [ ]:
import numpy as np
from hcve_lib.functional import t

def a(t):
    values = list(t.values())
    if isinstance(values[0], tuple):
        lower, higher = transpose_list(values)
        mean_ci_size = mean(np.array(higher)-np.array(lower))
        center = mean([mean(lower), mean(higher)])
        return [center-mean_ci_size/2, center+mean_ci_size/2]
    else:
        return mean(values)

mean_metrics_per_cohort = valmap(
    lambda per_test: valmap(a, transpose_dict(per_test)),
    metrics_per_cohort
)

mean_metrics_per_cohort_df = DataFrame()

for method_name, statistics in mean_metrics_per_cohort.items():
    try:
        print(f'{statistics=}')
        metrics_df_add = DataFrame({
            'method': [method_name],
            'mean': [statistics['mean']],
            'ci_low': [statistics['ci'][0]],
            'ci_high': [t(statistics['ci'][1]-statistics['mean'])],
            'std': [statistics['std']],
        })
        mean_metrics_per_cohort_df = pandas.concat([mean_metrics_per_cohort_df, metrics_df_add])
    except TypeError:
        print('ERROR')


fig = px.bar(mean_metrics_per_cohort_df, x='method', y='mean', error_y='ci_high', range_y=[0.5,1])
fig.update_layout(width=400)

In [ ]:
fig = px.bar(mean_metrics_per_cohort_df, x='method', y='mean', error_y='ci_high', range_y=[0.5,0.8])
fig.update_layout(width=400)

In [ ]:
px.bar(metrics_per_cohort_df, x='method', y='mean', error_y='ci_high', range_y=[0.5,0.1])

In [ ]:
metrics_per_cohort_df = DataFrame()

for method_name, method_metric_values in metrics_per_cohort.items():
    for test_cohort_name, metric_values in method_metric_values.items():
        try:
            metrics_df_add = DataFrame({
                'method': [method_name] * len(metric_values),
                'test_cohort': [test_cohort_name] * len(metric_values),
                'values': metric_values,
            })
            metrics_per_cohort_df = pandas.concat([metrics_per_cohort_df, metrics_df_add])
        except TypeError:
            print('ERROR')

### TODO: Separate evaluation

In [ ]:
group_results = {}

pcp_hf_result = load_run_results('229ef6c05f5d40acb57f5f62707f1e38')

for group_name, group_id in GROUPS.items():
    group = load_group_results(group_id)
    group_results[group_name] = group
    group['pcp_hf'] = pcp_hf_result

In [ ]:
group_results['coxnet'][0]

In [ ]:
metrics_per_cohort = valmap(
    lambda prediction: compute_metric_result(
        BootstrappedMetric(
            StratifiedMetric(CIndex(), splits={'30_to_80': list(get_30_to_80(X).index)}),
            RANDOM_STATE,
            iterations=ITERATIONS,
            return_summary=False,
        ),
        y,
        prediction,
    ),
    averaged_results,
)

In [ ]:
metrics_per_cohort_df = DataFrame()

for method_name, method_metric_values in metrics_per_cohort.items():
    for test_cohort_name, metric_values in method_metric_values.items():
        try:
            metrics_df_add = DataFrame({
                'method': [method_name] * len(metric_values),
                'test_cohort': [test_cohort_name] * len(metric_values),
                'values': metric_values,
            })
            metrics_per_cohort_df = pandas.concat([metrics_per_cohort_df, metrics_df_add])
        except TypeError:
            print('ERROR')

In [ ]:
fig = px.strip(
    metrics_per_cohort_df,
    x="method",
    y="values",
    color='test_cohort',
    stripmode='overlay',
    range_y=[0.5, 1]
)

fig.update_traces(marker_size=2, marker_opacity=0.9)
fig.update_layout(legend=dict(itemsizing='constant'))
fig.show()

# Single- vs multi-center / methods predictive performance comparison

In [ ]:
def get_lm_lco_metrix_matrix(
        metric: str,
        method: str,
        filter_string: str = None,
        show_info: bool = False,

):
    return get_metric_matrix(
        get_latest_chosen_run_group('optimized_nested_lm', method, filter_string),
        get_latest_chosen_run_group('optimized_nested_lco', method, filter_string),
        metric,
        show_info=show_info,

    )

def show_lco_lm_matrix_matrix(metric: str = 'c_index', *args, **kwargs) -> None:
    matrix = get_lm_lco_metrix_matrix(metric, *args, **kwargs)

    plot_metric_matrix(matrix)\
        .update_layout(title=metric)\
        .show()

def get_matrix_change(matrix1: DataFrame, matrix2: DataFrame) -> DataFrame:
    return DataFrame(get_numerical_matrix(matrix2)) - DataFrame(get_numerical_matrix(matrix1))

def show_matrix_change(matrix1: DataFrame, matrix2: DataFrame) -> DataFrame:
    return plot_matrix(get_matrix_change(matrix1, matrix2))

In [ ]:
show_lco_lm_matrix_matrix('c_index', 'gb')
show_lco_lm_matrix_matrix('c_index', 'coxnet')

In [ ]:

from deps.evaluation import get_subruns_in_group


def get_subrun_metrics(group_id: str):
    runs = get_subruns_in_group(group_id)

    return {run.data.tags['mlflow.runName']: run.data.metrics['c_index'] for run in runs}


In [ ]:
def get_lm_subruns(cohort_name, results, filter_out_bad: bool = False):
    return [
        c_index for split_name, c_index in results.items() if split_name[1] == cohort_name
        if c_index > 0.5 or not filter_out_bad
    ]



In [ ]:
from hcve_lib.functional import map_deep
from deps.evaluation import get_metric_matrix_lm
from typing import Tuple
from deps.evaluation import get_metric_matrix_lco


def get_lm_lco_bar(
        experiment_name: str,
        method_name: str,
        filter_string: str = None,
) -> Tuple[Series, Series]:
    lm_metrics = get_metric_matrix_lm(
        get_latest_chosen_run_group(experiment_name + '_lm', method_name, filter_string),
        'c_index'
    )

    lco_metrics = get_metric_matrix_lco(
        get_latest_chosen_run_group(experiment_name + '_lco', method_name, filter_string),
        'c_index'
    )

    lm = DataFrame(map_deep(lm_metrics, lambda item, _: item['mean'] if item else None, levels=2))\
        .mean(axis=1)

    lco = DataFrame(lco_metrics).loc['mean'].astype('float')
    return lm, lco

In [ ]:

lm_gb, lco_gb = get_lm_lco_bar('optimized_nested', 'gb', 'tags.subsample="False"')
lm_cn, lco_cn = get_lm_lco_bar('optimized_nested', 'coxnet', 'tags.subsample="False"')

COLOR_CN = '#9bbedb'
COLOR_SGB = '#ffbf7f'
FONT = 'Helvetica'
STRIPES_OPACITY = 0.4


In [ ]:

runs = DataFrame({
    'CN: Single-centre ': lm_cn,
    'CN: Multi-centre': lco_cn,
    'SGB: Single-centre': lm_gb,
    'SGB: Multi-centre': lco_gb,
}).loc[list(COHORTS)]


In [ ]:

fig = px.bar(
    runs,
    barmode="group",
    orientation='v',
    # color_discrete_sequence=[
    #     px.colors.qualitative.Dark24[0],
    #     px.colors.qualitative.Dark24[1],
    #     '#2e68e5',
    #     '#db488a',
    # ],
    height=400,
    width=1000,
    template='simple_white'
)


In [ ]:

# for num, label in ((0, 'SC'), (1, 'MC'), (2, 'SC'), (3, 'MC')):


In [ ]:

fig.data[0].marker.color = COLOR_CN

fig.data[1].marker.color = COLOR_CN

fig.data[1].marker.pattern.fgopacity = STRIPES_OPACITY

fig.data[1].marker.pattern.shape = '/'


In [ ]:

fig.data[2].marker.color = COLOR_SGB

fig.data[3].marker.color = COLOR_SGB

fig.data[3].marker.pattern.shape = '/'

fig.data[3].marker.pattern.fgopacity = STRIPES_OPACITY


In [ ]:

#

# fig.update_layout(legend=dict(

#     yanchor="top",

#     y=1.20,

#     xanchor="left",

#     x=0.01,

#     font={'size': 28},

# ))


In [ ]:

fig.update_xaxes(

    ticktext=COHORTS,

    tickvals=list(range(len(COHORTS))),

    tickfont=dict(size=22)

)

fig.update_layout(

    font=dict(family=FONT, size=28),

    bargroupgap=0.1,

    xaxis_title="Test cohort",

    yaxis_title="c-index",

    yaxis_range=[0.5, 0.85],

    legend={

        'title': {'text': None}

    },

    template='simple_white'

)

fig.update_yaxes(showgrid=True)

fig.write_image("./data/predictive_performance_lm_lco2.svg")

fig.write_image("./data/predictive_performance_lm_lco2.jpeg")

fig.show()


In [ ]:
from hcve_lib.tracking import search_latest_root_completed_run_children


def get_latest_subrun_metrics(experiment_name: str, method_name: str) -> Dict[str, float]:
    runs = search_latest_root_completed_run_children(experiment_name, method_name, "tags.chosen = 'True'")

    return {

        run.data.tags['mlflow.runName']: run.data.metrics['c_index']

        for run in runs

    }


In [ ]:

def get_and_plot_compare_experiments_averaged(

        method_names: List[str],

        experiment_name: str,

):
    metrics1 = get_latest_subrun_metrics(experiment_name, method_name)

    metrics2 = get_latest_subrun_metrics(experiment_name, method_name)

    fig = plot_compare_groups(metrics1, metrics2)

    fig.update_layout({'title': method_name})

    fig.show()


In [ ]:

def plot_compare_groups(metrics1, metrics2):
    fig = px.bar(

        DataFrame({

            'default': Series(metrics1),

            'optimized': Series(metrics2),

        }),

        barmode="group",

        orientation='v',

        height=400

    )

    fig.update_layout(

        xaxis_title="Cohort",

        yaxis_title="c-index",

        yaxis_range=[0.5, 1]

    )

    fig.update_yaxes(showgrid=True)

    return fig


Abstract
bar
chart

from typing import Dict, List

from plotly import express as px


In [ ]:

def get_latest_subrun_metrics(experiment_name: str, method_name: str) -> Dict[str, float]:
    runs = search_latest_root_completed_run_children(experiment_name, method_name, "tags.chosen = 'True'")

    return {

        run.data.tags['mlflow.runName']: run.data.metrics['c_index']

        for run in runs

    }


In [ ]:

def get_and_plot_compare_experiments_averaged(

        method_names: List[str],

        experiment_name: str,

):
    metrics1 = get_latest_subrun_metrics(experiment_name, method_name)

    metrics2 = get_latest_subrun_metrics(experiment2, method_name)

    fig = plot_compare_groups(metrics1, metrics2)

    fig.update_layout({'title': method_name})

    fig.show()


In [ ]:

def plot_compare_groups(metrics1, metrics2):
    fig = px.bar(

        DataFrame({

            'default': Series(metrics1),

            'optimized': Series(metrics2),

        }),

        barmode="group",

        orientation='v',

        height=400

    )

    fig.update_layout(

        xaxis_title="Cohort",

        yaxis_title="c-index",

        yaxis_range=[0.5, 1]

    )

    fig.update_yaxes(showgrid=True)

    return fig


In [ ]:

def get_lco_metrics(experiment_name: str, method_name: str):
    runs = get_lco(experiment_name, method_name)

    return {run.data.tags['mlflow.runName']: run.data.metrics['c_index'] for run in runs}


In [ ]:

def get_lco(experiment_name: str, method_name: str):

optimized_lco_id = get_experiment_by_name(experiment_name).experiment_id


In [ ]:

root_runs = mlflow.search_runs(

    optimized_lco_id,

    "tags.chosen = 'True' and tags.mlflow.runName = '" + method_name + "'",

    output_format='list',

    max_results=1,

)

if len(root_runs) == 0:

    return

else:

    root_run = root_runs[0]


In [ ]:

search_filter = f'tags.mlflow.parentRunId = "{root_run.info.run_id}"'


In [ ]:

return mlflow.search_runs(

    get_experiment_by_name(experiment_name).experiment_id,

    search_filter,

    output_format='list')


In [ ]:

c_index_sgb_nested = [get_lco_metrics('optimized_nested_lco', 'gb')[cohort] for cohort in COHORTS]

c_index_sgb = [get_lco_metrics('optimized_nested_lco', 'gb')[cohort] for cohort in COHORTS]

c_index_pcp_hf = [get_lco_metrics('default_lco', 'pc')[cohort] for cohort in COHORTS]


In [ ]:

runs = {

    'Survival Gradient Boosting (SGB)': c_index_sgb,

    'Pooled Cohort Equations (PCP-HF)': c_index_pcp_hf,

}

fig = px.bar(

    DataFrame(runs),

    barmode="group",

    orientation='v',

    color_discrete_sequence=px.colors.qualitative.Dark24,

    height=400,

    width=1000,

    template='simple_white'

)


In [ ]:

fig.update_layout(legend=dict(

    yanchor="top",

    y=1.18,

    xanchor="left",

    x=0.01,

    font={'size': 24}

))


In [ ]:

fig.update_xaxes(

    ticktext=COHORTS,

    tickvals=list(range(len(COHORTS))),

)


In [ ]:

fig.update_layout(

    font=dict(family="Times New Roman", size=23),

    xaxis_title="Test cohort",

    yaxis_title="c-index",

    yaxis_range=[0.5, 0.9],

    legend={

        'title': {'text': None}

    },

)

fig.update_yaxes(showgrid=True)

fig.write_image("./data/predictive_performance.svg")

fig.show()

